## All Qualifying Session Analysis

Using the data gathered in the previous file, I will conduct an analysis for all qualifying sessions & drivers in the 2021 season.

The data contains the fastest lap for each driver in each qualifying session in the 2021 season

In [1]:
#Setting up - import packages
import pandas as pd
import numpy as np
import fastf1 as ff1
#Enable the cache
ff1.Cache.enable_cache('C:/Users/jackh/OneDrive/Documents/Python Scripts/f1_cache') 

In [2]:
#Load existing data - saved as .csv file
raw = pd.read_csv('C:/Users/jackh/OneDrive/Documents/Python Scripts/2021_f1_qualifying_laps.csv')

In [3]:
raw.head()

['VER']


### I: Breaking Down Tracks into Minisectors

I will build an intra-team comparison across the season; however, before we analyze drivers we must break each track into minisectors, because telemetry data does not exactly line up at each point. By grouping each lap into minisectors we can analyze each teammate's performance on the same granularity.

In [4]:
#Define number of minisectors
num_minisectors = 25.0
#Find the max distance for each gp
distance_gp = raw.groupby('GP')['Distance'].max().reset_index().rename(columns = {"Distance":"max_distance"})
#Rename distance field
#distance_gp = distance_gp.rename(columns = {"Distance":"max_distance"})
#And attach to main df
df = raw.merge(distance_gp, how = 'inner', on = 'GP')

In [5]:
#Now calculate the minisector for each point in the data
df['Minisector'] = (df['Distance'] // (df['max_distance']/num_minisectors + .01) ) + 1

#And convert minisector to integer type - must do this in a second step
df['Minisector'] = df['Minisector'].astype(int)

### II: Assigning each driver an average speed per track & minisector

Because the telemetry isn't captured at exactly the same spot for each driver, a good way to break down performance is to capture average speed per minisector.

In [14]:
#First - create an object that ranks overall speed of each driver, so that we ensure each team's faster driver is the one that comes out ahead vs. his teammate (ie the gap in speed is always positive for the faster driver)
r = df[["Location", "Driver", "Speed"]]
r = r.groupby(['Location', 'Driver'])['Speed'].mean().reset_index()
r['rank'] = r['Speed'].rank(ascending = False)
r = r.drop('Speed', axis = 1)




In [ ]:
#And merge into the dataframe
df = df.merge(r, how = 'left', on = 'Driver')

In [17]:
#Add team information to dataset
driver_team = pd.DataFrame({'Driver':['VER', 'PER', 'HAM', 'BOT', 'LEC', 'SAI', 'RIC', 'NOR', 'ALO', 'OCO'
                                      , 'VET', 'STR', 'GAS', 'TSU', 'RUS', 'LAT', 'RAI', 'GIO', 'MSC', 'MAZ'],
                            'Team': ['Red Bull', 'Red Bull', 'Mercedes', 'Mercedes', 'Ferrari', 'Ferrari', 'McLaren', 'McLaren', 'Alpine', 'Alpine'
                                     , 'Aston Martin', 'Aston Martin', 'Alpha Tauri', 'Alpha Tauri', 'Williams', 'Williams', 'Alfa Romeo' , 'Alfa Romeo', 'Haas', 'Haas']})

df = df.merge(driver_team, how = 'left', on = 'Driver')

In [18]:
#Calculate the faster driver's gap to their teammate
average_speed = df.groupby(['GP', 'Location', 'Minisector', 'Team', 'Driver'])['Speed'].mean().reset_index().sort_values(by=['GP', 'Team', 'Minisector'], ascending = (True))

### JH NOTE: THIS SHOULD COME LATER, NOT NOW
Now that we have assigned each telemetry point to a minisector, we can assign a speed category to that minisector, based on each driver's average speed in that minisector

In [8]:
sector_speed = df.groupby(['GP', 'Minisector'])['Speed'].mean().reset_index()
low_speeds = sector_speed.groupby('GP')['Speed'].quantile(q=.33).rename('low_speed_threshold')
high_speeds = sector_speed.groupby('GP')['Speed'].quantile(q=.66).rename('high_speed_threshold')

speed_key = pd.concat([low_speeds, high_speeds], axis=1)

In [9]:
#Join thresholds to main df
df = df.merge(speed_key, how = 'left', on = 'GP')
#Assign each minisector a speed category based on all drivers' average speed through that minisector
df['Sector_speed'] = df['Speed'].apply(lambda x: 'Low' if x < df['low_speed_threshold'] else 'Medium' if x < df['high_speed_threshold'] else 'High')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().